In [0]:
import numpy as np

##Функция

In [0]:
def nest_skok(x):
    f = 0.25 * (x[0] - 1) ** 2 + np.sum([(x1 - 2 * x0 ** 2 + 1) ** 2 for (x1, x0) in zip(x[1:], x[:-2])])
    return f

In [0]:
def grad_nest_skok(x):
    df = 0.5 * (x[0] - 1) + np.sum([(2 - 8 * x0) * (x1 - 2 * x0 ** 2 + 1) for (x1, x0) in zip(x[1:], x[:-2])])
    return df

In [0]:
def grad2_nest_skok(x):
    d2f = 0.5 + np.sum([(2 - 8 * (x1 - 6 * (x0 ** 2) + 1)) for (x1, x0) in zip(x[1:], x[:-2])])
    return d2f

In [51]:
x = [-1, 1, 1]
print('x:', x)
print('f(x):', nest_skok(x))
print('df(x):', grad_nest_skok(x))

x: [-1, 1, 1]
f(x): 1.0
df(x): -1.0


##Градиентный спуск

In [0]:
def gradient_descent(lr, x0, steps):
    x_prev = x0
    for i in range(steps):
        x = x_prev - lr * grad_nest_skok(x_prev)
        x_prev = x
    return x

In [53]:
lr = 1e-4
x0 = [-1, 1, 1, 1]
steps = 100
print('Before:', x0, nest_skok(x0))
x = gradient_descent(lr, x0, steps)
print('After:', x, nest_skok(x))

Before: [-1, 1, 1, 1] 1.0
After: [-0.99273903  1.00726097  1.00726097  1.00726097] 0.9945417091479689


##Стандартный метод Гаусса-Ньютона

In [0]:
def gn_method(x0, steps):
    x_prev = x0
    for i in range(steps):
        x = x_prev - (grad2_nest_skok(x_prev)) ** (-1) * grad_nest_skok(x_prev)
        x_prev = x
    return x

In [55]:
x0 = [-1, 1, 1, 1]
steps = 100
print('Before:', x0, nest_skok(x0))
x = gn_method(x0, steps)
print('After:', x, nest_skok(x))

Before: [-1, 1, 1, 1] 1.0
After: [-0.98532751  1.01467249  1.01467249  1.01467249] 0.9926760182169204


##Модифицированный метод Гаусса-Ньютона

In [63]:
def f_tau(x, tau, M):
    fx = nest_skok(x)
    dfx = grad_nest_skok(x)
    dfx_t = dfx
    g = dfx_t * fx
    return tau + np.sqrt(fx ** 2) ** 2 / tau - (1 / (tau * dfx_t * dfx + tau ** 2 * M) * g * g)


def df_tau(x, tau, M):
    fx = nest_skok(x)
    dfx = grad_nest_skok(x)
    dfx_t = dfx
    g = dfx_t * fx
    return 1 - np.sqrt(fx ** 2) ** 2 / tau ** 2 - (1 / (dfx_t * dfx + 2 * tau * M) * g * g)


def h(x, tau, M):
    fx = nest_skok(x)
    dfx = grad_nest_skok(x)
    dfx_t = dfx
    return (-1 / M) * dfx_t *  (1 / (tau + 1 / M * dfx * dfx_t)) * fx


def search_tau(x, tau, M, steps=100):
    tau_prev = tau
    for k in range(steps):
        tau = tau_prev - f_tau(x, tau_prev, M) / df_tau(x, tau_prev, M)
        tau_prev = tau
    return tau


def get_vmk(x, M):
    tau = 0.1
    tau = search_tau(x, tau, M)
    h_cur = h(x, tau, M)
    vmk = np.sqrt((x - h_cur) ** 2)
    return vmk


def fm(x, y, M):
    return np.sqrt(np.sum((nest_skok(x) + grad_nest_skok(x) * (y - x)) ** 2)) + 0.5 * M * np.sum((y - x) ** 2)


def mod_gauss(x0, L0, L, steps=100):
    Mks = np.linspace(L0, 2*L)
    x_prev = x0
    x = x_prev
    for k in range(steps):
        for Mk in Mks:
            vmk = get_vmk(x_prev, Mk)
            if fm(vmk, vmk, Mk) <= fm(x_prev, vmk, Mk):
                x = vmk
                x_prev = x
                fl = True
                break
    return x


x0 = [-1, 1, 1]
L = 1e4
L0 = L / 2
x = mod_gauss(x0, L0, L)
print(x)
print(nest_skok(x))

[1.02763107 1.02751045 1.02751045]
0.007338010915004839


##Метод трех квадратов

##Адаптивные варианты методов